In [1]:
import os 
import sys
import pygwalker as pyg
import pandas as pd

In [2]:

#Importing Configs
# Define the path where config.py is located
os.chdir('/Users/kylenabors/Documents/GitHub/Finance-ML-Modeling')
config_file_path = os.getcwd()
print(config_file_path)

# Add this path to the sys.path
sys.path.append(config_file_path)

import config

#Configs
database_file = config.database
database_folder = config.database_folder
bert_models = config.bert_models
bert_models_local = config.bert_models_local
Sentiment_models = config.Sentiment_models

Body = config.Body
Model = config.Model
Model_Subfolder = f'/{Body} Texts/{Model}'
#Model_Folder = config.texts
Model_Folder = config.Sentiment_models
Model_Folder = Model_Folder + Model_Subfolder
Model_Folder = config.Sentiment_models

Body_2 = config.Body_2
Model_2 = config.Model_2
Model_Subfolder_2 = f'/{Body_2} Texts/{Model_2}'
Model_Folder_2 = config.texts
Model_Folder_2 = Model_Folder_2 + Model_Subfolder_2

/Users/kylenabors/Documents/GitHub/Finance-ML-Modeling
/Users/kylenabors/Documents/GitHub/Finance-ML-Modeling
/Users/kylenabors/Documents
/Users/kylenabors/Documents/GitHub/Finance-ML-Modeling


In [3]:
df = pd.read_csv(f"{Sentiment_models}/{Body}_{Model}_advanced_sentiment_texts.csv")
df_2 = pd.read_csv(f"{Sentiment_models}/{Body_2}_{Model_2}_advanced_sentiment_texts.csv")

In [4]:
dummies = pd.get_dummies(df['sentiment'], dtype=float)
df = df.join(dummies)
dummies_2 = pd.get_dummies(df_2['sentiment'], dtype=float)
df_2 = df_2.join(dummies)

In [5]:
print(df.head())
print(df_2.head())

   Unnamed: 0        date                                title  \
0           0  2023-08-31  Disinflation and the Phillips curve   
1           1  2023-08-31  Disinflation and the Phillips curve   
2           2  2023-08-31  Disinflation and the Phillips curve   
3           3  2023-08-31  Disinflation and the Phillips curve   
4           4  2023-08-31  Disinflation and the Phillips curve   

              type                                            segment  \
0  Isabel Schnabel  speech disinflation and the phillips curve spe...   
1  Isabel Schnabel  activity has moderated visibly , and forwardlo...   
2  Isabel Schnabel  but important pockets of resilience remain , e...   
3  Isabel Schnabel  headline inflation has come down , mainly on t...   
4  Isabel Schnabel  but underlying price pressures remain stubborn...   

  sentiment  confidence  neg  pos  
0       pos         0.8  0.0  1.0  
1       neg         0.6  1.0  0.0  
2       pos         1.0  0.0  1.0  
3       pos         

In [6]:

df_group = df[['date', 'pos', 'neg']]
df_group['monthly'] = df_group['date'].str[0:7]
df_group = df_group[['monthly', 'pos', 'neg']]
df_group = df_group.groupby('monthly', as_index = False).mean()
df_group['net'] = df_group['pos'] - df_group['neg']

df_2_group = df_2[['date', 'pos', 'neg']]
df_2_group['monthly'] = df_2_group['date'].str[0:7]
df_2_group = df_2_group[['monthly', 'pos', 'neg']]
df_2_group = df_2_group.groupby('monthly', as_index = False).mean()
df_2_group['net'] = df_2_group['pos'] - df_group['neg']

df_merged = df_group.merge(df_2_group, on='monthly', how='inner')
df_merged.fillna(0, inplace=True)

/var/folders/9h/v8gnzwrs73v52x4lyzz2z2zc0000gn/T/ipykernel_78181/1400928521.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_group['monthly'] = df_group['date'].str[0:7]
/var/folders/9h/v8gnzwrs73v52x4lyzz2z2zc0000gn/T/ipykernel_78181/1400928521.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2_group['monthly'] = df_2_group['date'].str[0:7]


In [7]:
print(df_merged.head())

df_merged.rename(columns={'pos_x': 'pos_1', 'neg_x': 'neg_1', 'net_x': 'net_1', 'pos_y': 'pos_2', 'neg_y': 'neg_2', 'net_y': 'net_2'}, inplace=True)

print(df_merged.head())

   monthly     pos_x     neg_x     net_x     pos_y     neg_y     net_y
0  2002-06  0.890966  0.109034  0.781931  0.818627  0.181373  0.709593
1  2002-07  0.841026  0.158974  0.682051  0.867470  0.132530  0.708496
2  2002-08  0.853503  0.146497  0.707006  0.838710  0.161290  0.692213
3  2002-09  0.814815  0.185185  0.629630  0.813648  0.186352  0.628463
4  2002-10  0.887006  0.112994  0.774011  0.826498  0.173502  0.713504
   monthly     pos_1     neg_1     net_1     pos_2     neg_2     net_2
0  2002-06  0.890966  0.109034  0.781931  0.818627  0.181373  0.709593
1  2002-07  0.841026  0.158974  0.682051  0.867470  0.132530  0.708496
2  2002-08  0.853503  0.146497  0.707006  0.838710  0.161290  0.692213
3  2002-09  0.814815  0.185185  0.629630  0.813648  0.186352  0.628463
4  2002-10  0.887006  0.112994  0.774011  0.826498  0.173502  0.713504


In [8]:
df_merged['net_diff'] = df_merged['net_1'] - df_merged['net_2']

In [10]:
funds = pd.read_excel('/Users/kylenabors/Documents/Database/Other Data/FedFundsRate.xlsx', sheet_name='Monthly')
print(funds.head())
energy = pd.read_csv('/Users/kylenabors/Documents/Database/Other Data/Energy Prices/US Energy CPI.csv')
print(energy.head())

        DATE  FEDFUNDS Date Adjusted
0 1999-12-31      5.45    2000-01-01
1 2000-01-31      5.73    2000-02-01
2 2000-02-29      5.85    2000-03-01
3 2000-03-31      6.02    2000-04-01
4 2000-04-30      6.27    2000-05-01
         DATE  CPIENGSL
0  2000-01-01     115.0
1  2000-02-01     118.8
2  2000-03-01     124.3
3  2000-04-01     120.9
4  2000-05-01     120.0


In [ ]:
funds['Diff'] = funds['FEDFUNDS'].diff()


In [ ]:
df_merged.to_csv(f"{Sentiment_models}/{Body}_{Model}_vs_{Body_2}_{Model_2}_advanced_sentiment_texts.csv", index=False)

In [11]:
import numpy as np
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

net_1 = df_merged['net_1'].to_list()
net_2 = df_merged['net_2'].to_list()


model = sm.OLS(net_1, net_2)
results = model.fit()

In [12]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.971
Model:                            OLS   Adj. R-squared (uncentered):              0.971
Method:                 Least Squares   F-statistic:                              3694.
Date:                Sat, 21 Oct 2023   Prob (F-statistic):                    4.92e-87
Time:                        13:23:03   Log-Likelihood:                          82.282
No. Observations:                 112   AIC:                                     -162.6
Df Residuals:                     111   BIC:                                     -159.8
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             1.0216      0.017     60.776      0.000       0.988       1.055
==============================================================================
Omnibus:                       12.626   Durbin-Watson:                   1.994
Prob(Omnibus):                  0.002   Jarque-Bera (JB):               39.695
Skew:                          -0.007   Prob(JB):                     2.40e-09
Kurtosis:                       5.916   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
gwalker = pyg.walk(df_merged)

Box(children=(HTML(value='<div id="ifr-pyg-0" style="height: auto">\n    <head>\n        <meta http-equiv="Con…

In [2]:
s = "acadae"

res = ''.join(sorted(s))
print(res)

aaacde


In [ ]:
|